In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [5]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [6]:
answers = {}

In [7]:
# Some data structures that will be useful

In [8]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [9]:
len(allRatings)

200000

In [10]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))

In [ ]:
##################################################
# Rating prediction (CSE258 only)                #
##################################################

In [ ]:
### Question 9

In [ ]:
answers['Q9'] = validMSE

In [ ]:
assertFloat(answers['Q9'])

In [ ]:
### Question 10

In [ ]:
answers['Q10'] = [maxUser, minUser, maxBeta, minBeta]

In [ ]:
assert [type(x) for x in answers['Q10']] == [str, str, float, float]

In [ ]:
### Question 11

In [ ]:
answers['Q11'] = (lamb, validMSE)

In [ ]:
assertFloat(answers['Q11'][0])
assertFloat(answers['Q11'][1])

In [ ]:
predictions = open("predictions_Rating.csv", 'w')
for l in open("pairs_Rating.csv"):
    if l.startswith("userID"): # header
        predictions.write(l)
        continue
    u,b = l.strip().split(',') # Read the user and item from the "pairs" file and write out your prediction
    # (etc.)
    
predictions.close()

In [11]:
##################################################
# Read prediction                                #
##################################################

In [12]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [13]:
print(allRatings[0])

('u67805239', 'b61372131', 4)


In [14]:
### Question 1

In [15]:
valid = [[d[0], d[1], 1] for d in ratingsValid]

books = set()
booksPerUser = defaultdict(set)
usersPerBook = defaultdict(set)
for u,b,r in readCSV("train_Interactions.csv.gz"):
    usersPerBook[b].add(u)
    booksPerUser[u].add(b)
    books.add(b)

userValid = [d[0] for d in valid]
for u in userValid:
    noReadBooks = books - booksPerUser[u]
    randomBook = random.choice(list(noReadBooks))
    valid.append([u, randomBook, 0])

xValid = [[d[0], d[1]] for d in valid]
yValid = [d[2] for d in valid]

prediction = []
for d in xValid:
    if d[1] in return1:
        prediction.append(1)
    else:
        prediction.append(0)

predValid = numpy.array(prediction) == numpy.array(yValid)
acc1 = sum(predValid) / len(predValid)

In [16]:
answers['Q1'] = acc1
answers['Q1']

0.7074

In [17]:
assertFloat(answers['Q1'])

In [18]:
### Question 2

In [19]:
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

acc2 = 0
threshold = 0
for thre in range(1, 101):
    return1 = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalRead * thre * 0.01: break

    prediction = []
    for d in xValid:
        if d[1] in return1:
            prediction.append(1)
        else:
            prediction.append(0)

    predValid = numpy.array(prediction) == numpy.array(yValid)
    accThre = sum(predValid) / len(predValid)
 
    if accThre > acc2:
        acc2 = accThre
        threshold = thre * 0.01

In [20]:
answers['Q2'] = [threshold, acc2]
answers['Q2']

[0.72, 0.7509]

In [21]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [22]:
### Question 3/4

In [23]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

def mostSimilar(u, b):
    similarities = []
    books = booksPerUser[u]
    for book in books:
        if book == b: continue
        sim = Jaccard(usersPerBook[b], usersPerBook[book])
        similarities.append(sim)
    similarities.sort(reverse=True)
    return similarities[:10]
    
acc3 = 0
threshold = 0
for thre in range(1, 11):
    similarList = []
    prediction = 0
    tempYValid = []
    for d in valid:
        similarList = mostSimilar(d[0], d[1])

        if len(similarList)==0: continue
        tempYValid.append(d[2])

        if similarList[0] > thre * 0.1:
            prediction += (d[2]==1)
        else:
            prediction += (d[2]==0)

    #predValid = numpy.array(prediction) == numpy.array(tempYValid)
    accThre = prediction / len(valid)
 
    if accThre > acc3:
        acc3 = accThre
        threshold = thre * 0.1

acc3

0.5098

In [24]:
return1 = set()
count = 0
thresholdPopular = answers['Q2'][0]
thresholdJaccard = threshold

for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead * thresholdPopular: break

prediction = []
tempYValid = []
for d in valid:
    similarList = mostSimilar(d[0], d[1])

    if len(similarList)==0: continue
    tempYValid.append(d[2])

    if d[1] in return1 or similarList[0] > thresholdJaccard:
        prediction.append(1)
    else:
        prediction.append(0)

predValid = numpy.array(prediction) == numpy.array(tempYValid)
acc4 = sum(predValid) / len(predValid)
acc4

0.7606769138336754

In [25]:
answers['Q3'] = acc3
answers['Q4'] = acc4
print(answers['Q3'])
print(answers['Q4'])

0.5098
0.7606769138336754


In [26]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [27]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    # (etc.)
    similarList = mostSimilar(u, b)

    if len(similarList)==0:
        predictions.write(u + "," + b + "," + "0\n")
        continue

    if b in return1 or similarList[0] > thresholdJaccard:
        predictions.write(u + "," + b + "," + "1\n")
    else:
        predictions.write(u + "," + b + "," + "0\n")

predictions.close()

In [28]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [29]:
assert type(answers['Q5']) == str

In [30]:
##################################################
# Category prediction (CSE158 only)              #
##################################################

In [31]:
### Question 6

In [32]:
data = []

for d in readGz("train_Category.json.gz"):
    data.append(d)

In [33]:
data[0]

{'user_id': 'u75242413',
 'review_id': 'r45843137',
 'rating': 4,
 'review_text': "a clever book with a deeply troubling premise and an intriguing protagonist. Thompson's clean, sparse prose style kept each page feeling light even as some rather heavy existential questions dropped upon them. I enjoyed it. \n and that cover design is boom-pow gorgeous.",
 'n_votes': 1,
 'genre': 'mystery_thriller_crime',
 'genreID': 3}

In [34]:
trainCategory = data[:90000]
validCategory = data[90000:]

wordCount = defaultdict(int)
punctuation = set(string.punctuation)
totalWords = 0
for d in trainCategory:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        totalWords += 1
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [35]:
answers['Q6'] = counts[:10]
answers['Q6']

[(666039, 'the'),
 (402958, 'and'),
 (354159, 'a'),
 (336144, 'to'),
 (328560, 'i'),
 (292310, 'of'),
 (197178, 'is'),
 (191392, 'in'),
 (184705, 'it'),
 (174671, 'this')]

In [36]:
assert [type(x[0]) for x in answers['Q6']] == [int]*10
assert [type(x[1]) for x in answers['Q6']] == [str]*10

In [37]:
### Question 7

In [38]:
words = [x[1] for x in counts[:1000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data]
y = [d['genreID'] for d in data]

In [39]:
Xtrain = X[:9*len(X)//10]
ytrain = y[:9*len(y)//10]
Xvalid = X[9*len(X)//10:]
yvalid = y[9*len(y)//10:]

In [40]:
model = linear_model.LogisticRegression()
model.fit(Xtrain, ytrain)
predictions = model.predict(Xvalid)
correct = predictions == yvalid
acc7 = sum(correct) / len(correct)

In [41]:
answers['Q7'] = acc7
answers['Q7']

0.6407

In [42]:
assertFloat(answers['Q7'])

In [43]:
### Question 8

In [44]:
X_train = X[:8*len(X)//10]
y_train = y[:8*len(y)//10]
X_valid = X[8*len(X)//10:]
y_valid = y[8*len(y)//10:]

In [45]:
model = linear_model.LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
correct = predictions == y_valid
acc8 = sum(correct) / len(correct)
acc8

0.641

In [46]:
answers['Q8'] = acc8

In [47]:
assertFloat(answers['Q8'])

In [48]:
# Run on test set
dataSet = []
for d in readGz("test_Category.json.gz"):
    dataSet.append(d)

Xdata = [feature(d) for d in dataSet]
predData = model.predict(Xdata)
predData

array([3, 3, 0, ..., 2, 2, 2])

In [49]:
predictions = open("predictions_Category.csv", 'w')
pos = 0

index = 0
for l in open("pairs_Category.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    # (etc.)
    predictions.write(u + ',' + b + ',' + str(predData[index]) + '\n')
    index += 1

predictions.close()

In [50]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()